In [ ]:
import pandas as pd
from utils import util

In [2]:
video_num = 25
window_size = 90
shift_size = 90
threshold = 0.1
label_path = '/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/emo_and_au-gaze-hp(video1-25)_ver2-gazesign.csv'

sequence_list = pd.DataFrame()
sequence_list_nomixed = pd.DataFrame()

all_info = {
    'video_name': [],
    'size': [],
    '0': [],
    '1': [],
    '2': []
}

nomixed_info = {
    'video_name': [],
    'size': [],
    '0': [],
    '1': [],
    '2': []
}

for i in range(video_num):
    video_name = 'video' + str(i+1)
    _seq_list = util.get_sequence_list(video_name=video_name, window_size=window_size, shift_size=shift_size, threshold=threshold, label_path=label_path)
    _seq_list_nomixed = util.get_sequence_list(video_name=video_name, window_size=window_size, shift_size=shift_size, threshold=threshold, drop_mixed=True, label_path=label_path)
    
    all_info['video_name'].append(video_name)
    all_info['size'].append(len(_seq_list))
    all_info['0'].append(len(_seq_list[_seq_list['emotion'] == 0]))
    all_info['1'].append(len(_seq_list[_seq_list['emotion'] == 1]))
    all_info['2'].append(len(_seq_list[_seq_list['emotion'] == 2]))
    
    nomixed_info['video_name'].append(video_name)
    nomixed_info['size'].append(len(_seq_list_nomixed))
    nomixed_info['0'].append(len(_seq_list_nomixed[_seq_list_nomixed['emotion'] == 0]))
    nomixed_info['1'].append(len(_seq_list_nomixed[_seq_list_nomixed['emotion'] == 1]))
    nomixed_info['2'].append(len(_seq_list_nomixed[_seq_list_nomixed['emotion'] == 2]))
    
    sequence_list = pd.concat([sequence_list, _seq_list], axis=0)
    sequence_list_nomixed = pd.concat([sequence_list_nomixed, _seq_list_nomixed], axis=0)
    
sequence_list = sequence_list.reset_index(drop=True)
sequence_list_nomixed = sequence_list_nomixed.reset_index(drop=True)

all_info = pd.DataFrame(all_info)
all_info.loc['total'] = all_info.iloc[:, 1:].sum()
all_info = pd.concat([all_info.iloc[:, 0], all_info.iloc[:, 1:].astype(int)], axis=1)

nomixed_info = pd.DataFrame(nomixed_info)
nomixed_info.loc['total'] = nomixed_info.iloc[:, 1:].sum()
nomixed_info = pd.concat([nomixed_info.iloc[:, 0], nomixed_info.iloc[:, 1:].astype(int)], axis=1)

In [3]:
all_info

,video_name,size,0,1,2
0,video1,84,84,0,0
1,video2,13,13,0,0
2,video3,289,289,0,0
3,video4,537,537,0,0
4,video5,115,115,0,0
5,video6,115,115,0,0
6,video7,59,59,0,0
7,video8,286,286,0,0
8,video9,187,187,0,0
9,video10,309,309,0,0


In [4]:
nomixed_info

,video_name,size,0,1,2
0,video1,64,64,0,0
1,video2,0,0,0,0
2,video3,269,269,0,0
3,video4,517,517,0,0
4,video5,95,95,0,0
5,video6,95,95,0,0
6,video7,39,39,0,0
7,video8,266,266,0,0
8,video9,167,167,0,0
9,video10,289,289,0,0


In [4]:
sequence_list.to_csv(f'/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/seq_labels(video1-25)_ver2-gazesign_wsize{window_size}-ssize{shift_size}-th{threshold:.0e}.csv', index=False)

In [ ]:
sequence_list_nomixed.to_csv(f'/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/nomixed_seq_labels(video1-25)_ver2-gazesign_wsize{window_size}-ssize{shift_size}.csv', index=False)

In [27]:
# drop hard negative samples
target_emo = 2

seq_list = pd.read_csv(f'/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/seq_labels(video1-25)_ver2-gazesign_wsize{window_size}-ssize{shift_size}-th{threshold:.0e}.csv')
seq_list['video_name'], _ = zip(*seq_list['img_path'].map(util.get_video_name_and_frame_num))
# if target_emo == 1, video_name = 'video1' ~ 'video17' elif target_emo == 2, video_name = 'video18' ~ 'video25'
target_video_list = [f'video{i}' for i in range(1, 18)] if target_emo == 1 else [f'video{i}' for i in range(18, 26)]

# if video_name isin target_video_list, drop emotion == 0 and negative_rate != 1.0
seq_list = seq_list[~((seq_list['video_name'].isin(target_video_list)) & (seq_list['emotion'] == 0) & (seq_list['negative_rate'] != 1.0))]
seq_list = seq_list.reset_index(drop=True)
seq_list = seq_list.drop(columns=['video_name'])
seq_list.to_csv(f'/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/drop_discomfort_hardnegative_seq_labels(video1-25)_ver2-gazesign_wsize{window_size}-ssize{shift_size}-th{threshold:.0e}.csv', index=False)
    